In [1]:
from multiprocessing import Lock, Process, Queue, current_process
import time
import queue # imported for using queue.Empty exception


def do_job(tasks_to_accomplish, tasks_that_are_done):
    while True:
        try:
            '''
                try to get task from the queue. get_nowait() function will 
                raise queue.Empty exception if the queue is empty. 
                queue(False) function would do the same task also.
            '''
            task = tasks_to_accomplish.get_nowait()
        except queue.Empty:

            break
        else:
            '''
                if no exception has been raised, add the task completion 
                message to task_that_are_done queue
            '''
            print(task)
            tasks_that_are_done.put(task + ' is done by ' + current_process().name)
            time.sleep(.5)
    return True


def main():
    number_of_task = 10
    number_of_processes = 4
    tasks_to_accomplish = Queue()
    tasks_that_are_done = Queue()
    processes = []

    for i in range(number_of_task):
        tasks_to_accomplish.put("Task no " + str(i))

    # creating processes
    for w in range(number_of_processes):
        p = Process(target=do_job, args=(tasks_to_accomplish, tasks_that_are_done))
        processes.append(p)
        p.start()

    # completing process
    for p in processes:
        p.join()

    # print the output
    while not tasks_that_are_done.empty():
        print(tasks_that_are_done.get())

    return True


In [2]:
main()

Task no 0
Task no 1Task no 2

Task no 3
Task no 4
Task no 5Task no 6

Task no 7
Task no 8
Task no 9
Task no 0 is done by Process-1
Task no 1 is done by Process-2
Task no 2 is done by Process-3
Task no 3 is done by Process-4
Task no 4 is done by Process-1
Task no 5 is done by Process-2
Task no 6 is done by Process-3
Task no 7 is done by Process-4
Task no 8 is done by Process-1
Task no 9 is done by Process-2


True

In [15]:
from multiprocessing import Lock, Process, Queue, current_process
import time
import queue # imported for using queue.Empty exception
import glob
import os
import sys
from datetime import datetime

def get_subdirs(path, q_done):
    # get all subdirs
    #all_subdirs = []
    for root,subdirs,files in os.walk(path, topdown=True):
        #print(root,subdirs,files)
        for d in subdirs:
            #all_files.append(root+"/"+f)
            q_done.put(os.path.realpath(os.path.join(root, d)))
    #return all_subdirs

def worker(q_dirs, q_done):
    while True:
        try:
            '''
                try to get task from the queue. get_nowait() function will 
                raise queue.Empty exception if the queue is empty. 
                queue(False) function would do the same task also.
            '''
            dir_path = q_dirs.get_nowait()
            paths = get_subdirs(dir_path, q_done)
        except queue.Empty:
            break
        else:
            '''
                if no exception has been raised, add the task completion 
                message to task_that_are_done queue
            '''
            print(dir_path)
            q_done.put(paths)
            time.sleep(5)
    return True

def stuff():
    number_of_task = 10
    number_of_processes = 4
    q_dirs = Queue()
    q_done = Queue()
    processes = []
    path = "/home/maugb/aptude/"
    list_of_dirs = glob.glob(path+"*")

    for d in list_of_dirs:
        q_dirs.put(d)

    # creating processes
    for w in range(number_of_processes):
        p = Process(target=worker, args=(q_dirs, q_done))
        processes.append(p)
        p.start()

    # completing process
    for p in processes:
        p.join()

    # print the output
    all_subdirs = set()
    while not q_done.empty():
        all_subdirs.add(q_done.get_nowait())
    print(all_subdirs)

    return True


In [16]:
stuff()

/home/maugb/aptude/archive_files.py
/home/maugb/aptude/cosas.sql
/home/maugb/aptude/django_test
/home/maugb/aptude/Building-Django-2.0-Web-Applications
/home/maugb/aptude/flask_test
/home/maugb/aptude/shit.py
/home/maugb/aptude/somos_kudasai_elasticsearch
/home/maugb/aptude/stuff.sh
/home/maugb/aptude/tests.py
/home/maugb/aptude/test
/home/maugb/aptude/__pycache__
{'/mnt/c/Users/MauricioIlhuicaminaG/aptude/Building-Django-2.0-Web-Applications/Chapter07/django/cueeneh/management/__pycache__', '/mnt/c/Users/MauricioIlhuicaminaG/aptude/somos_kudasai_elasticsearch/somos_kudasai_elasticsearch', '/mnt/c/Users/MauricioIlhuicaminaG/aptude/Building-Django-2.0-Web-Applications/.git/objects/2f', '/mnt/c/Users/MauricioIlhuicaminaG/aptude/Building-Django-2.0-Web-Applications/.git/objects/27', '/mnt/c/Users/MauricioIlhuicaminaG/aptude/Building-Django-2.0-Web-Applications/.git/refs', '/mnt/c/Users/MauricioIlhuicaminaG/aptude/somos_kudasai_elasticsearch/tests', '/mnt/c/Users/MauricioIlhuicaminaG/aptud

True